In [1]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import as_dtype, dtFactor, lookup, desc_col
from pa_lib.util import cap_words, collect, value, normalize_rows

# Load data sets

In [2]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    global_codes = load_pickle("code_ratios.pkl")

15:39:52 [INFO] Started loading binary file ...
15:39:52 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
15:39:52 [INFO] ... finished loading binary file in 0.3s (0.97s CPU)
15:39:52 [INFO] Started loading pickle file ...
15:39:52 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
15:39:52 [INFO] ... finished loading pickle file in 0.01s (0.02s CPU)
15:39:52 [INFO] Started loading pickle file ...
15:39:52 [INFO] Reading from file C:\Users\kpf\data\axinova\code_ratios.pkl
15:39:52 [INFO] ... finished loading pickle file in 0.01s (0.0s CPU)


# Function to look up code ratios for a given station and variable

In [36]:
def _code_pivot(df, values, time_scale):
    return df.pivot_table(
        values=values,
        index=["DayOfWeek", time_scale],
        columns="Code",
        aggfunc="mean",
        fill_value=0,
    )


def _global_ratios(variable, time_scale):
    ratios = global_codes[time_scale].query("Variable == @variable")
    return _code_pivot(ratios, "Ratio", time_scale)


def ax_var_ratios(station, variable, time_scale="Hour"):
    subset = ax_data.query(f"Station == @station and Variable == @variable")
    actual_ratios = normalize_rows(_code_pivot(subset, "Value", time_scale))
    expected_ratios = _global_ratios(variable, time_scale)
    return (actual_ratios, expected_ratios)


def as_percent(df):
    return (df * 100).round(1)

In [37]:
(actual, expected) = ax_var_ratios(
    station="Zürich HB", variable="md_ek", time_scale="Hour"
)

display(as_percent(actual))
display(as_percent(expected))
display(as_percent(actual - expected))

Code            Keine Angabe  Mehr als 12'000 CHF  Weniger als 3'000 CHF  \
DayOfWeek Hour                                                             
Monday    00             0.0                 16.9                   16.7   
          01             0.0                  0.0                    0.0   
          02             0.0                  0.0                    0.0   
          03             0.0                  0.0                    0.0   
          04             0.0                  0.0                    0.0   
...                      ...                  ...                    ...   
Sunday    19             8.7                 17.4                    8.2   
          20            12.4                 18.5                   10.7   
          21            16.6                 18.7                    9.1   
          22             9.7                 11.9                   11.7   
          23            17.7                 13.7                   16.3   

Code            Zwischen 3'000 und 4'500 CHF  Zwischen 4'501 und 6'000 CHF  \
DayOfWeek Hour                                                               
Monday    00                            15.4                          22.1   
          01                             0.0                           0.0   
          02                             0.0                           0.0   
          03                             0.0                           0.0   
          04                             0.0                           0.0   
...                                      ...                           ...   
Sunday    19                            12.7                          14.9   
          20                            10.5                          15.5   
          21                            11.4                          10.5   
          22                            15.5                          14.6   
          23                            10.5                          14.0   

Code            Zwischen 6'001 und 9'000 CHF  Zwischen 9'001 und 12'000 CHF  
DayOfWeek Hour                                                               
Monday    00                            15.3                           13.6  
          01                             0.0                          100.0  
          02                           100.0                            0.0  
          03                           100.0                            0.0  
          04                             0.0                          100.0  
...                                      ...                            ...  
Sunday    19                            15.3                           22.9  
          20                            16.6                           15.7  
          21                            15.4                           18.5  
          22                            20.0                           16.6  
          23                            12.4                           15.4  

[168 rows x 7 columns]

Code            Keine Angabe  Mehr als 12'000 CHF  Weniger als 3'000 CHF  \
DayOfWeek Hour                                                             
Monday    00             5.2                 27.6                    4.2   
          01            30.0                 13.6                    0.0   
          02             4.8                  6.0                    0.0   
          03             0.0                 56.0                    0.0   
          04             1.8                 37.0                    0.0   
...                      ...                  ...                    ...   
Sunday    19            12.4                 18.7                    7.6   
          20            16.4                 19.0                    6.1   
          21            15.9                 20.8                    5.0   
          22            12.0                 18.4                    5.0   
          23            13.6                 21.0                    6.8   

Code            Zwischen 3'000 und 4'500 CHF  Zwischen 4'501 und 6'000 CHF  \
DayOfWeek Hour                                                               
Monday    00                             3.1                          20.4   
          01                             2.0                           2.4   
          02                            51.2                           0.0   
          03                            14.4                          11.8   
          04                             7.1                           4.3   
...                                      ...                           ...   
Sunday    19                             6.6                          11.6   
          20                             8.0                           9.4   
          21                            10.1                           7.1   
          22                            12.6                          12.5   
          23                             8.0                          14.1   

Code            Zwischen 6'001 und 9'000 CHF  Zwischen 9'001 und 12'000 CHF  
DayOfWeek Hour                                                               
Monday    00                            24.5                           15.0  
          01                            22.6                           29.4  
          02                            24.3                           13.7  
          03                             8.3                            9.5  
          04                            46.0                            3.7  
...                                      ...                            ...  
Sunday    19                            20.6                           22.6  
          20                            17.7                           23.4  
          21                            16.3                           24.7  
          22                            17.8                           21.7  
          23                            20.3                           16.2  

[168 rows x 7 columns]

Code            Keine Angabe  Mehr als 12'000 CHF  Weniger als 3'000 CHF  \
DayOfWeek Hour                                                             
Monday    00            -5.2                -10.7                   12.5   
          01           -30.0                -13.6                    0.0   
          02            -4.8                 -6.0                    0.0   
          03             0.0                -56.0                    0.0   
          04            -1.8                -37.0                    0.0   
...                      ...                  ...                    ...   
Sunday    19            -3.7                 -1.3                    0.7   
          20            -4.0                 -0.5                    4.6   
          21             0.7                 -2.2                    4.1   
          22            -2.4                 -6.5                    6.7   
          23             4.1                 -7.2                    9.5   

Code            Zwischen 3'000 und 4'500 CHF  Zwischen 4'501 und 6'000 CHF  \
DayOfWeek Hour                                                               
Monday    00                            12.3                           1.7   
          01                            -2.0                          -2.4   
          02                           -51.2                           0.0   
          03                           -14.4                         -11.8   
          04                            -7.1                          -4.3   
...                                      ...                           ...   
Sunday    19                             6.1                           3.2   
          20                             2.5                           6.2   
          21                             1.3                           3.3   
          22                             2.9                           2.1   
          23                             2.5                          -0.1   

Code            Zwischen 6'001 und 9'000 CHF  Zwischen 9'001 und 12'000 CHF  
DayOfWeek Hour                                                               
Monday    00                            -9.2                           -1.5  
          01                           -22.6                           70.6  
          02                            75.7                          -13.7  
          03                            91.7                           -9.5  
          04                           -46.0                           96.3  
...                                      ...                            ...  
Sunday    19                            -5.3                            0.3  
          20                            -1.1                           -7.7  
          21                            -0.9                           -6.3  
          22                             2.2                           -5.1  
          23                            -7.9                           -0.8  

[168 rows x 7 columns]